In [20]:
import boto3
import io
import configparser
import psycopg2
import pandas as pd
import io

In [38]:
config = configparser.ConfigParser()
config.read('../config.ini')

aws_access_key_id = config['AWS']['aws_access_key_id']
aws_secret_access_key = config['AWS']['aws_secret_access_key']
region_name = config['AWS']['region_name']
rds_db_endpoint = config['AWS']['rds_db_endpoint']
rds_db_port = config['AWS']['rds_db_port']
rds_db_user = config['AWS']['rds_db_user']
rds_db_name = config['AWS']['rds_db_name']
rds_db_password = config['AWS']['rds_db_password']



bucket_name = config['AWS-S3']['bucket_name']
bucket_file_users = config['AWS-S3']['bucket_file_users']
bucket_file_ratings = config['AWS-S3']['bucket_file_ratings']
bucket_file_books = config['AWS-S3']['bucket_file_books']


In [4]:
def new_aws_session(aws_access_key_id, aws_secret_access_key, region): 
    session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region)
    return session

In [5]:
session = new_aws_session(aws_access_key_id, aws_secret_access_key, region_name)

session

Session(region_name='eu-west-3')

In [15]:
# Connection to RDS 

try:
    conn = psycopg2.connect(f"host={rds_db_endpoint} port={rds_db_port}  user={rds_db_user}  password={rds_db_password} dbname={rds_db_name}")
    # conn = psycopg2.connect("host= localhost port=5432  user=postgres  password=anamac dbname=db_book_recommendation")
except psycopg2.Error as e :
    print(e)

In [21]:
sql_users = '''
    SELECT 
        userID ,
        age ,
        location_city ,
        location_department ,
        location_country 
    FROM USERS
'''
try:
    result_users = pd.read_sql(sql_users, conn)
    df_users = pd.DataFrame(result_users)
    df_users
except psycopg2.Error as e :
    print(e)

/var/folders/pv/gcq6fn3s0_l1qn6qdtx7srf00000gn/T/ipykernel_26890/913844295.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_users = pd.read_sql(sql_users, conn)


In [44]:
sql_books = '''
    SELECT 
        isbn ,
        book_title ,
        book_author ,
        year_of_publication ,
        publisher 
    FROM BOOKS
'''
try:
    result_books = pd.read_sql(sql_books, conn)
    df_books = pd.DataFrame(result_books)
    df_books
except psycopg2.Error as e :
    print(e)

/var/folders/pv/gcq6fn3s0_l1qn6qdtx7srf00000gn/T/ipykernel_26890/1304569003.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_books = pd.read_sql(sql_books, conn)


In [35]:
sql_ratings = '''
    SELECT 
        userid,
        isbn ,
        book_rating
    FROM RATINGS
'''
try:
    result_ratings = pd.read_sql(sql_ratings, conn)
    df_ratings = pd.DataFrame(result_ratings)
    df_ratings
except psycopg2.Error as e :
    print(e)

/var/folders/pv/gcq6fn3s0_l1qn6qdtx7srf00000gn/T/ipykernel_26890/1697284052.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_ratings = pd.read_sql(sql_ratings, conn)


In [22]:
# Create a bucket to store all datasets
s3 = session.client('s3', region_name=region_name)
bucket_name = bucket_name
location = {'LocationConstraint': region_name}
response = s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)


In [39]:
# Get in-stream buffer for the dataset
csv_buffer= io.StringIO()


In [40]:
# Upload csv to s3 bucket
df_users.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket_name, Key=bucket_file_users, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '8E9J3SBD438X6T6T',
  'HostId': 'uoT1dpW5oYPhIC8Msykz9qDTkthB5w7McVWmipOnws5C8wf8ThjFuaPC6F3m1lx4EvKXypYTQqJUCDPSqz3qCw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'uoT1dpW5oYPhIC8Msykz9qDTkthB5w7McVWmipOnws5C8wf8ThjFuaPC6F3m1lx4EvKXypYTQqJUCDPSqz3qCw==',
   'x-amz-request-id': '8E9J3SBD438X6T6T',
   'date': 'Thu, 09 Nov 2023 15:00:02 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f5fc391b03afdb6b8ad4f15b9fdf6622"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f5fc391b03afdb6b8ad4f15b9fdf6622"',
 'ServerSideEncryption': 'AES256'}

In [45]:
# Upload csv to s3 bucket
csv_buffer.close()
csv_buffer= io.StringIO()
df_books.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket_name, Key=bucket_file_books, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'RKZ7A292WSJ2PTQ2',
  'HostId': 'pwypXAPBaHu6YxGsU6wCebwPP+ea5QG11ZnXH5lA7UiRCYLXS0MNhSOzSM4rwQm70242pTCVSc8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pwypXAPBaHu6YxGsU6wCebwPP+ea5QG11ZnXH5lA7UiRCYLXS0MNhSOzSM4rwQm70242pTCVSc8=',
   'x-amz-request-id': 'RKZ7A292WSJ2PTQ2',
   'date': 'Thu, 09 Nov 2023 15:07:15 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"233ba5768cb7ab9fd98f1e3ad74ce087"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"233ba5768cb7ab9fd98f1e3ad74ce087"',
 'ServerSideEncryption': 'AES256'}

In [42]:
# Upload csv to s3 bucket
csv_buffer.close()
csv_buffer= io.StringIO()
df_ratings.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket_name, Key=bucket_file_ratings, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '6AWECZT7Z5BF5YH4',
  'HostId': 'hytcXoQSvYk3P02KOtEyF6rI5bSXgsNaFPJM9b5aDG1w1/3yuFQGonAVlEy7qVYlteZxKobWdOA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hytcXoQSvYk3P02KOtEyF6rI5bSXgsNaFPJM9b5aDG1w1/3yuFQGonAVlEy7qVYlteZxKobWdOA=',
   'x-amz-request-id': '6AWECZT7Z5BF5YH4',
   'date': 'Thu, 09 Nov 2023 15:00:52 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"fea4945ff0040d671092bdac6b22b71d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fea4945ff0040d671092bdac6b22b71d"',
 'ServerSideEncryption': 'AES256'}